In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings_small.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from scipy import stats
from ast import literal_eval

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')

# **SIMPLE RECOMMENDER/ Demographic Filtering**



This type of recommenders offers based on movie popularity and genre. This recommender does not give personalized recommendations based on the user history. All we have to do is sort our movies based on ratings and popularity and display the top movies of our list

In [3]:
sr=pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
sr.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
sr.shape

(45466, 24)

In [5]:
np.sum(sr.isna())

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [6]:
sr['genres'] = sr['genres'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
sr = sr.drop([19730, 29503, 35587])

#  Use the Weighted Rating Formula

I will use IMDB's weighted rating formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) =  (v/(v+m))*R   +   (m/(v+m))*C



* v is the number of votes for the movie 
* m is the minimum votes required to be listed in the chart
* R is the average rating of the movie
* C is the mean vote across the whole report

In [7]:
#Number of votes for the movie
number_votes=sr[sr['vote_count'].notnull()]['vote_count']
number_votes

0        5415.0
1        2413.0
2          92.0
3          34.0
4         173.0
          ...  
45461       1.0
45462       3.0
45463       6.0
45464       0.0
45465       0.0
Name: vote_count, Length: 45460, dtype: float64

In [8]:
#Average rating of the movie
vote_average=sr[sr['vote_average'].notnull()]['vote_average']
vote_average

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
45461    4.0
45462    9.0
45463    3.8
45464    0.0
45465    0.0
Name: vote_average, Length: 45460, dtype: float64

In [9]:
#Mean votes across the whole report
C=np.mean(vote_average)
C

5.618207215133889

The minimum votes required to be on the list can be defined by taking 90% percentile as our cutoff.

In [10]:
m=np.quantile(number_votes,0.9)
m

160.0

In [11]:
Reduced_movies=sr[(sr['vote_count']>m) & (sr['vote_count'].notnull())][['title','popularity','genres','vote_count','vote_average']]
Reduced_movies.head()

,title,popularity,genres,vote_count,vote_average
0,Toy Story,21.9469,"[Animation, Comedy, Family]",5415.0,7.7
1,Jumanji,17.0155,"[Adventure, Fantasy, Family]",2413.0,6.9
4,Father of the Bride Part II,8.38752,[Comedy],173.0,5.7
5,Heat,17.9249,"[Action, Crime, Drama, Thriller]",1886.0,7.7
8,Sudden Death,5.23158,"[Action, Adventure, Thriller]",174.0,5.5


In [12]:
np.sum(Reduced_movies.isna())
#There is no null values in the final data set

title           0
popularity      0
genres          0
vote_count      0
vote_average    0
dtype: int64

In [13]:
Reduced_movies['Weighted_rating'] = (Reduced_movies['vote_count']/(Reduced_movies['vote_count']+m))*Reduced_movies['vote_average'] + (m/(Reduced_movies['vote_count']+m))*C
Reduced_movies.head()

,title,popularity,genres,vote_count,vote_average,Weighted_rating
0,Toy Story,21.9469,"[Animation, Comedy, Family]",5415.0,7.7,7.640253
1,Jumanji,17.0155,"[Adventure, Fantasy, Family]",2413.0,6.9,6.820293
4,Father of the Bride Part II,8.38752,[Comedy],173.0,5.7,5.660700
5,Heat,17.9249,"[Action, Crime, Drama, Thriller]",1886.0,7.7,7.537201
8,Sudden Death,5.23158,"[Action, Adventure, Thriller]",174.0,5.5,5.556626


In [14]:
#Sort the values in descending order
Reduced_movies=Reduced_movies.sort_values('Weighted_rating',ascending=False)
Reduced_movies.head(20)

,title,popularity,genres,vote_count,vote_average,Weighted_rating
314,The Shawshank Redemption,51.6454,"[Drama, Crime]",8358.0,8.5,8.445869
834,The Godfather,41.1093,"[Drama, Crime]",6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,34.457,"[Comedy, Drama, Romance]",661.0,9.1,8.421453
12481,The Dark Knight,123.167,"[Drama, Action, Crime, Thriller]",12269.0,8.3,8.265477
2843,Fight Club,63.8696,[Drama],9678.0,8.3,8.256385
292,Pulp Fiction,140.95,"[Thriller, Crime]",8670.0,8.3,8.251406
522,Schindler's List,41.7251,"[Drama, History, War]",4436.0,8.3,8.206639
23673,Whiplash,64.3,[Drama],4376.0,8.3,8.205404
5481,Spirited Away,41.0489,"[Fantasy, Adventure, Animation, Family]",3968.0,8.3,8.196055
2211,Life Is Beautiful,39.395,"[Comedy, Drama]",3643.0,8.3,8.187171


In [15]:
def simple_recommendation(data,genre='Nothing',percentile=0.9):
    if genre != 'Nothing':
        data=data[data['genres'].apply(lambda x: True if genre in x else False)]
    votes=data[data['vote_count'].notnull()]['vote_count']
    vote_average=data[data['vote_average'].notnull()]['vote_average']
    C=np.mean(vote_average)
    m=np.quantile(votes,percentile)
    data=data[(data['vote_count']>m) & (data['vote_count'].notnull())]
    data['Weighted_rating'] = (data['vote_count']/(data['vote_count']+m))*data['vote_average'] + (m/(data['vote_count']+m))*C
    return data.sort_values('Weighted_rating',ascending=False)

In [16]:
simple_recommendation(sr,'Nothing')[['title','popularity','genres','vote_count','vote_average']].head(20)

,title,popularity,genres,vote_count,vote_average
314,The Shawshank Redemption,51.6454,"[Drama, Crime]",8358.0,8.5
834,The Godfather,41.1093,"[Drama, Crime]",6024.0,8.5
10309,Dilwale Dulhania Le Jayenge,34.457,"[Comedy, Drama, Romance]",661.0,9.1
12481,The Dark Knight,123.167,"[Drama, Action, Crime, Thriller]",12269.0,8.3
2843,Fight Club,63.8696,[Drama],9678.0,8.3
292,Pulp Fiction,140.95,"[Thriller, Crime]",8670.0,8.3
522,Schindler's List,41.7251,"[Drama, History, War]",4436.0,8.3
23673,Whiplash,64.3,[Drama],4376.0,8.3
5481,Spirited Away,41.0489,"[Fantasy, Adventure, Animation, Family]",3968.0,8.3
2211,Life Is Beautiful,39.395,"[Comedy, Drama]",3643.0,8.3


# **SIMPLE RECOMMENDATION FINAL SYSTEM WITH Genre**

In [17]:
simple_recommendation(sr,'Mystery')[['title','popularity','genres','vote_count','vote_average']].head(20)

,title,popularity,genres,vote_count,vote_average
15480,Inception,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",14075.0,8.1
46,Se7en,18.4574,"[Crime, Mystery, Thriller]",5915.0,8.1
4099,Memento,15.4508,"[Mystery, Thriller]",4168.0,8.1
11354,The Prestige,16.9456,"[Drama, Mystery, Thriller]",4510.0,8.0
23675,Gone Girl,154.801,"[Mystery, Thriller, Drama]",6023.0,7.9
877,Rear Window,17.9113,"[Drama, Mystery, Thriller]",1531.0,8.2
14825,Shutter Island,15.8136,"[Drama, Thriller, Mystery]",6559.0,7.8
897,2001: A Space Odyssey,22.4946,"[Science Fiction, Mystery, Adventure]",3075.0,7.9
9430,Oldboy,10.6169,"[Drama, Thriller, Mystery, Action]",2000.0,8.0
4748,Donnie Darko,18.3031,"[Fantasy, Drama, Mystery]",3574.0,7.7


# **CONTENT BASED RECOMMENDER**

The content of the movie (overview, cast, crew, keyword, tagline etc) is used to find its similarity with other movies. Then the movies that are most likely to be similar are recommended.



**MOVIE DESCRIPTION/PLOT BASED RECOMMENDER**

In [18]:
sr.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

# # As the computation is very higher if we use all 40,000 data, so we will take percentile to be 80% from the previous Recommeder and then use it here for the Description based Recommender.

In [19]:
sample=simple_recommendation(sr,'Nothing',0.70)
sample['overview']=sample['overview'].fillna('')
sample['tagline']=sample['tagline'].fillna('')
sample['description']=(sample['tagline']+' '+sample['overview']).fillna('')

In [20]:
vector=TfidfVectorizer(stop_words='english',analyzer='word',ngram_range=(1,2))
matrix=vector.fit_transform(sample['description'])
matrix.shape

(13498, 367549)

In [21]:
cosine = linear_kernel(matrix, matrix)

In [22]:
data=sample.reset_index()
titles=data['title']
indices = pd.Series(data.index, index=data['title'])

In [23]:
def content_based_recommendation(title,indices=indices,titles=titles,cosine=cosine):
    idx=indices[title]
    sim_scores = list(enumerate(cosine[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    movie_indices=[i[0] for i in sim_scores[1:26]]
    return titles.iloc[movie_indices]

In [24]:
content_based_recommendation('Inception')

3696                                                Cypher
10191                                            The Brave
4642                                           Renaissance
11406                                             Hollywoo
13079                                     The Monkey's Paw
3586                                             Lone Star
13034                                             Altitude
1392                                                 Crumb
9560                                                 House
278                       What Ever Happened to Baby Jane?
2562                                      The Pink Panther
8839                                                  Cobb
7114                                               The Den
10381                                           Stone Cold
12825                                           Shadow Man
10376                                             Vendetta
13228                                             Cyborg

# # As our recommender only recommends based on the Plot or description of the movie, but many of us prefers movies based on actors or other crew members.

So, the next one will be based on Content based recommendations based on directors, actors and others

In [25]:
keyword=pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
credits=pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keyword['id'] = keyword['id'].astype('int')
credits['id'] = credits['id'].astype('int')
sr['id'] = sr['id'].astype('int')
sr = sr.merge(credits, on='id')
sr = sr.merge(keyword, on='id')


In [26]:
sample=simple_recommendation(sr,'Nothing',0.70)
sample.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords',
       'Weighted_rating'],
      dtype='object')

In [27]:
sample['cast'] = sample['cast'].apply(literal_eval)
sample['crew'] = sample['crew'].apply(literal_eval)
sample['keywords'] = sample['keywords'].apply(literal_eval)

In [28]:
#one of the important crew members we preferred for watching the film are DIRECTORS

In [29]:
def diro(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return ''

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [30]:
sample['director']=sample['crew'].apply(diro)
sample['cast']=sample['cast'].apply(get_list)
sample['keywords']=sample['keywords'].apply(get_list)

In [31]:
data = sample
def clean_data(x):
    if isinstance(x,list):
        return [str.lower(i.replace(" ","")) for i in x]
    else:
        if isinstance(x,str):
            #the director is multiplied with 3 to increase its weight as there are 3 actors.
            return [str.lower(x.replace(" ","")) for i in range(3)] 

features=['director','keywords','cast','genres']
for i in features:
    data[i]=data[i].apply(clean_data)
        

In [32]:
data['All']=(data['director'] + data['keywords'] + data['cast'] + data['genres']).apply(lambda x: ' '.join(x))
data.head()[features+['All']]

,director,keywords,cast,genres,All
10397,"[adityachopra, adityachopra, adityachopra]",[musical],"[shahrukhkhan, kajol, amrishpuri]","[comedy, drama, romance]",adityachopra adityachopra adityachopra musical...
314,"[frankdarabont, frankdarabont, frankdarabont]","[prison, corruption, policebrutality]","[timrobbins, morganfreeman, bobgunton]","[drama, crime]",frankdarabont frankdarabont frankdarabont pris...
841,"[francisfordcoppola, francisfordcoppola, franc...","[italy, loveatfirstsight, lossoffather]","[marlonbrando, alpacino, jamescaan]","[drama, crime]",francisfordcoppola francisfordcoppola francisf...
41418,"[makotoshinkai, makotoshinkai, makotoshinkai]","[supernatural, romance, school]","[ryunosukekamiki, monekamishiraishi, masaminag...","[romance, animation, drama]",makotoshinkai makotoshinkai makotoshinkai supe...
40253,"[alastairfothergill, alastairfothergill, alast...","[miniseries, greatcinematpgraphy]",[davidattenborough],[documentary],alastairfothergill alastairfothergill alastair...


In [33]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
matrix = count.fit_transform(data['All'])
matrix.shape

(13728, 113432)

In [34]:
cosine = cosine_similarity(matrix, matrix)
data = data.reset_index()
titles=data['title']
indices = pd.Series(data.index, index=data['title'])

In [35]:
content_based_recommendation('The Dark Knight',indices,titles,cosine)

330        The Dark Knight Rises
467                Batman Begins
66                  The Prestige
1357                   Following
4863                   Doodlebug
4865                   Doodlebug
2623                    Insomnia
487                      Dunkirk
35                  Interstellar
41                       Memento
34                     Inception
9652     Kidnapping Mr. Heineken
3114                    Thursday
4724                The Gauntlet
8588                      Twelve
11268           Hitman: Agent 47
1373                    Kick-Ass
2659        The Long Good Friday
2885                Metro Manila
3431                 Harry Brown
6752                     Hackers
7570                   Initial D
8807                      Hitman
3859           Kill the Irishman
4066        A Better Tomorrow II
Name: title, dtype: object

In [36]:
content_based_recommendation('Inception',indices,titles,cosine)

4863                                 Doodlebug
4865                                 Doodlebug
35                                Interstellar
41                                     Memento
66                                The Prestige
2623                                  Insomnia
5                              The Dark Knight
330                      The Dark Knight Rises
467                              Batman Begins
1357                                 Following
487                                    Dunkirk
8760                                   Don Jon
3601                                    Looper
9313                       L: Change the World
1397                           Minority Report
10116    Sky Captain and the World of Tomorrow
925                Mad Max 2: The Road Warrior
1515                                 The Abyss
3102                         Starship Troopers
4254                                Armageddon
6445                  X-Men Origins: Wolverine
6583         

**As, we can see the recommendations are very good based on the content which invloves the directors, actors and others.**